# 라이브러리

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import os

# RAG 기본 파이프라인

### 문서 로드

In [3]:
loader = PyMuPDFLoader("/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
docs[1].metadata

문서의 페이지수: 23


{'source': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 1,
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13462',
 'producer': 'Hancom PDF 1.3.0.542',
 'creationDate': "D:20231208132838+09'00'",
 'modDate': "D:20231208132838+09'00'",
 'trapped': ''}

### 문서 분할

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 72


In [7]:
print(docs[1].page_content)

2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5
   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11
   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달

In [8]:
print(split_documents[1].page_content)

2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5


### 임베딩 생성

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda:1'}  # CPU에서 실행
)

/home/woongjae/new_anaconda3/envs/rag-chatbot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### DB 생성 및 저장

In [6]:
# 벡터 스토어가 이미 있으면 로드, 없으면 새로 생성
if os.path.exists("./chroma_db_test"):
    vectorstore = Chroma(persist_directory="./chroma_db_test", embedding_function=embeddings)
else:
    vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_test")

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})
vectorstore.add_documents(split_documents)

/tmp/ipykernel_4158853/3840083469.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_test")


['be75e58d-8f8f-4621-9d47-f1cefe746a4d',
 'd98f880b-1896-4560-a85c-95932580ab5e',
 '0714a2ca-8fd7-4651-b326-10916d5b3520',
 'e1fc5dba-3c81-4351-8173-cfdde3c6a890',
 'd5f3f5ef-ee1d-4014-b900-79dbf2c66c52',
 'ef160c36-18f5-45ce-8b83-b0ff63c8353d',
 'eaddf79a-5f7a-4eef-92d8-3a37cf78e720',
 'ef88dc22-767e-4379-bc15-baee40f12309',
 '86679c92-e448-4940-a8f5-c5c31b0568c2',
 '4b41e192-8d3e-4022-b541-56f42dc34a3e',
 '02ff05cc-5cb7-4597-90bd-91734a2e1485',
 '58435be7-b5f0-410f-9371-7bec49a6a9fb',
 'a25a55bb-a351-42dd-858d-86bea41f7641',
 'c9b680fa-a344-4de9-9260-1c708096bb3d',
 '07385a29-3c54-4e34-b099-7d4729f14e9c',
 '2ddb17d5-1277-43a2-b6eb-e48e50ec4dea',
 'c89e0fd1-fc97-4f9b-876f-560289fdedb0',
 'b4af76fb-8656-4868-b402-e98a306bf7f3',
 '73ec8ca5-195b-4bb1-acd1-0efed062aeea',
 'ca092b34-a090-4d18-b793-a00003b836c9',
 'f7d86ae6-696a-4403-b0e3-1733977201c5',
 '0c238e7c-93ba-4b30-a8ca-542a84710fd0',
 'a98b84ea-eb58-4366-810f-a55835e6a298',
 'ae495e36-4149-4043-a17c-b4ba71843e8b',
 '92deb21a-13b8-

### 검색기(Retriever) 생성

In [7]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x72fc60231d90>, search_kwargs={})

In [8]:
retriever.invoke("AI소프트웨어 시장이 2027년에 달성할것 같은 매출액이 얼마일 것 같아?")

[Document(metadata={'author': 'dj', 'creationDate': "D:20231208132838+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231208132838+09'00'", 'page': 22, 'producer': 'Hancom PDF 1.3.0.542', 'source': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'subject': '', 'title': '', 'total_pages': 23, 'trapped': ''}, page_content='홈페이지 : https://spri.kr/\n보고서와 관련된 문의는 AI정책연구실(jayoo@spri.kr, 031-739-7352)으로 연락주시기 바랍니다.'),
 Document(metadata={'author': 'dj', 'creationDate': "D:20231208132838+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231208132838+09'00'", 'page': 2, 'producer': 'Hancom PDF 1.3.0.542', 'source': '/home/woongjae/rag-chatbot/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'subject': '', 'title': '', 't

### 프롬프트 생성

In [27]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)